In [1]:
#import libraries
from sklearn.datasets import load_boston # dataset
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip
import statsmodels.stats.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
import statsmodels as sm
import statsmodels.formula.api as smf
from sklearn import linear_model
import seaborn as sns

%matplotlib inline

/usr/local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# instantiate dataset and create DataFrame 
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
target = pd.DataFrame(boston.target, columns=["MEDV"])

In [3]:
# Basic exploratory analyses 
print("Rows,Columns:",df.shape)
df.describe()

Rows,Columns: (506, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.593761,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.596783,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.647423,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [4]:
# split dataset into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(df, target, test_size=0.30)


In [5]:
# Fitting using stats model
model_1 = smf.OLS(Y_train,X_train).fit()
prediction_1 = model_1.predict(X_train) # predict y_train values
prediction_2 = model_1.predict(X_test)# predict y_test values

In [6]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     596.2
Date:                Sat, 05 May 2018   Prob (F-statistic):          1.86e-225
Time:                        10:36:54   Log-Likelihood:                -1067.8
No. Observations:                 354   AIC:                             2162.
Df Residuals:                     341   BIC:                             2212.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0457      0.044     -1.042      0.298      -0.132       0.041
ZN             0.0591      0.018      3.329      0.001       0.024       0.094
INDUS          0.0216      0.079      0.275      0.783      -0.133       0.176
CHAS           2.1568      1.099      1.963      0.050      -0.005       4.318
NOX           -1.2258      4.162     -0.295      0.769      -9.411       6.960
RM             5.6223      0.369     15.223      0.000       4.896       6.349
AGE           -0.0031      0.016     -0.191      0.849      -0.035       0.029
DIS           -1.0464      0.236     -4.441      0.000      -1.510      -0.583
RAD            0.2058      0.088      2.337      0.020       0.033       0.379
TAX           -0.0126      0.005     -2.423      0.016      -0.023      -0.002
PTRATIO       -0.2682      0.134     -2.007      0.046      -0.531      -0.005
B              0.0154      0.003      4.885      0.000       0.009       0.022
LSTAT         -0.4938      0.061     -8.037      0.000      -0.615      -0.373
==============================================================================
Omnibus:                      155.130   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              838.036
Skew:                           1.789   Prob(JB):                    1.05e-182
Kurtosis:                       9.634   Cond. No.                     8.67e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.67e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Looking at the Summary

In the summary we see some interesting results. The INDUS coefficient (proportion of non-retail business acres per town) is not nearly statistically significant at the 5% or 10% level. The same goes for the AGE and NOX variables.

There seems to be no autocorrelation (as expected as this is not a time-series problem).

Overall, the Adjusted R^2 score is great, but how about RMSE (root mean square error), lets have a loo

In [7]:
rmse_training = sqrt(mean_squared_error(Y_train, prediction_1))
print("RMSE for training data:", rmse_training)
rmse_test = sqrt(mean_squared_error(Y_test,prediction_2))
print("RMSE for test data:", rmse_test)

RMSE for training data: 4.94107101649613
RMSE for test data: 4.967825294277917


## Checking for Multicollinearity


Hmm... The RMSE for the test data is quite low and very close to that of the training data. But can we do better?! Lets try by doing a correlation test to identify spurious variables. **A VIF Test.**

In [8]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_test.values, i) for i in range(X_test.shape[1])]
vif["features"] = X_test.columns
vif

,VIF Factor,features
0,1.746325,CRIM
1,3.071110,ZN
2,14.490036,INDUS
3,1.222059,CHAS
4,75.127789,NOX
5,89.547803,RM
6,25.876999,AGE
7,16.345562,DIS
8,11.812340,RAD
9,47.882192,TAX


There is no hard and fast rules on VIF factors, while a score above 10 is undesirable as it suggests very correlation that artificially inflates our R score, a subjective call must be taken depending on the problem. For example, while the TAX variable (full-value property-tax rate per 10,000 dollars) is highly correlated with the rest of the variables, it is too important of a variable to ignore, plus it is very significant at the 5% level.

NOX and INDUS on the other hand, we can try and remove and see if we get a better model. Let's re-create our training sets.

In [9]:
df_no_rm = df.copy()

In [10]:

#create second training set
df_no_rm = df_no_rm.drop(["RM"], axis = 1)
X_train_no_rm, X_test_no_rm, Y_train, Y_test = train_test_split(df_no_rm, target, test_size=0.30)
print(df.shape)
print(X_test_no_rm.shape)

(506, 13)
(152, 12)


In [11]:
# fit second model
model_2 = smf.OLS(Y_train,X_train_no_rm).fit()
prediction_3 = model_2.predict(X_train_no_rm)# predict y_test values
prediction_4 = model_2.predict(X_test_no_rm)# predict y_test values


In [12]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.928
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     369.8
Date:                Sat, 05 May 2018   Prob (F-statistic):          1.27e-187
Time:                        10:38:32   Log-Likelihood:                -1161.4
No. Observations:                 354   AIC:                             2347.
Df Residuals:                     342   BIC:                             2393.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0780      0.047     -1.645      0.101      -0.171       0.015
ZN             0.1315      0.021      6.209      0.000       0.090       0.173
INDUS         -0.2044      0.098     -2.077      0.039      -0.398      -0.011
CHAS           2.4227      1.472      1.646      0.101      -0.473       5.318
NOX           32.5113      4.786      6.793      0.000      23.097      41.925
AGE            0.0415      0.021      1.943      0.053      -0.001       0.084
DIS           -0.4180      0.302     -1.384      0.167      -1.012       0.176
RAD            0.0847      0.105      0.805      0.421      -0.122       0.292
TAX           -0.0118      0.006     -1.910      0.057      -0.024       0.000
PTRATIO        0.5375      0.156      3.439      0.001       0.230       0.845
B              0.0230      0.004      5.228      0.000       0.014       0.032
LSTAT         -0.8422      0.072    -11.749      0.000      -0.983      -0.701
==============================================================================
Omnibus:                       69.570   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              130.072
Skew:                           1.072   Prob(JB):                     5.69e-29
Kurtosis:                       5.054   Cond. No.                     7.75e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.75e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
rmse_training_2 = sqrt(mean_squared_error(Y_train, prediction_3))
print("RMSE for training data:", rmse_training)
rmse_test_2 = sqrt(mean_squared_error(Y_test,prediction_4))
print("RMSE for test data:", rmse_test)

RMSE for training data: 4.94107101649613
RMSE for test data: 4.967825294277917


In [18]:
lm = linear_model.LinearRegression()
model_2 = lm.fit(X_train,Y_train)

/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [19]:

rms = sqrt(mean_squared_error(Y_test, prediction_2))
print("RMSE_test:",rms)
rms_1 = sqrt(mean_squared_error(Y_train,prediction_1))
print("RMSE_train:",rms_1)

RMSE_test: 5.165986234287241
RMSE_train: 4.817268734240217


In [20]:
print(type(Y_test))

<class 'pandas.core.frame.DataFrame'>


In [21]:
residual = Y_test.values-prediction_2
#print(residual)
for i in X_test:
    plt.scatter(X_test[i],residual)
    plt.xlim(0,)
    plt.xlabel(i)
    plt.ylabel("residuals")
    plt.show()


Exception: Data must be 1-dimensional

## Test for Heteroskedacticity (Breush-Pagan)

Here we are testing for non-constant variance for our error.This violation is most common in cross=sectional data, rarely occuring on purely time-series data.  The example that I had given in class was of comparing state expenditures on Education. So, here MP and UP were large states and chandigah and J&K were small states so we had a non-constant variance of our error terms. One remedy would be to split the states by population and have seperate regressions for states over 10 crore indivuduals and less than 10 crore. 

The breusch pagan test is a hypothesis test, where our Null is that variance is constant and alternative is that variance is non-constant. Therefore our decision hinges on a p-value. 

In [22]:
import statsmodels

name = [ 'p-value']
test = statsmodels.stats.diagnostic.het_breuschpagan(model_1.resid, model_1.model.exog)
lzip(name, test)

[('p-value', 73.91031062416761)]

Here, we cannot reject the null. We are safe from this potential violation.